In [1]:
!pip install bert_score

import torch
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from bert_score import score

test_df = pd.read_csv("test_processed.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [2]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google-t5/t5-large")

device = torch.device("cuda") # GPU usage
model_t5 = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large")
model_t5.to(device)

MAX_ALLOWED_SEQUENCE = 8192 

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
def ReadingWithJumpingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < MAX_ALLOWED_SEQUENCE:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE,\
                                num_beams=4, early_stopping=True)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    JUMP = 100
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(int(start + MAX_ALLOWED_SEQUENCE + JUMP), len(words))
        chunks.append(' '.join(words[int(start):int(start + MAX_ALLOWED_SEQUENCE/2)]) \
                        + " " + ' '.join(words[int(start + MAX_ALLOWED_SEQUENCE/2) + JUMP:end]))
        start += MAX_ALLOWED_SEQUENCE/2

    print(len(chunks))
    
    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE,\
                                num_beams=4, early_stopping=True)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()
    
    summarized_text = ' '.join(summarized_chunks)
    
    if len(summarized_text.split()) > MAX_ALLOWED_SEQUENCE:
        return ReadingWithJumpingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=MAX_ALLOWED_SEQUENCE,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=MAX_ALLOWED_SEQUENCE,\
                                num_beams=4, early_stopping=True)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

The following test was done to assess the performance on the model on texts that it can analyze without splitting it in chunks

In [4]:
F1_t5_less_8192 = []

for index, row in test_df[test_df['reference_tokens_preprocessed'] < MAX_ALLOWED_SEQUENCE].iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingWithJumpingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t5_less_8192.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t5_less_8192.npy', F1_t5_less_8192)

sum = 0
for _ in F1_t5_less_8192:
    sum += _
print(sum / len(F1_t5_less_8192))

4996
eu regulation establishes uniform rule on marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on cross-border marketing of aifs and ucits.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

T5 BertScore F1: 0.79
1346
a member state shall submit to the commission statistic on all the aquaculture activity conducted in freshwater and saltwater on their territory. member state shall use survey or other statistically validated method covering at least 90 % of the total production by volume. the remaining part of the total production may be estimated.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.84
4077
application for a certificate shall be lodged within six month of the date on which the authorisation to place the product on the market is granted. application for an extension of the duration of a certificate already granted shall be lodged with the authority referred to in article 9 ( 1 ) member state may require that the certificate be subject to the payment of an annual fee.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
0.815039294797021


The following test was done to assess the performance on the model on texts that it must split in chunks to perform the summarization task

In [5]:
F1_t5_more_8192 = []

for index, row in test_df[test_df['reference_tokens_preprocessed'] > MAX_ALLOWED_SEQUENCE].iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens_preprocessed"])

    result_summary = ReadingWithJumpingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t5_more_8192.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t5_more_8192.npy', F1_t5_more_8192)

sum = 0
for _ in F1_t5_more_8192:
    sum += _
print(sum / len(F1_t5_more_8192))

14373
4
market surveillance authority shall ensure that only compliant products are made available on the market. they shall carry out their duty independently, impartially and without bias. market surveillance authority shall ensure that product presenting a serious risk are withdrawn or recalled.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
11101
3
in the digital sector, only the following action shall be eligible to receive financial support under this regulation. the amount of financial support shall not exceed 50 % of the total eligible cost. the maximum co-funding rate applicable to action shall be 85 %.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.79
21083
6
horizon europe is a framework programme for research and innovation. it aims to strengthen the scientific, technological, economic and societal impact of the union's investment in r & i. eit is a centrally managed one-stop shop for implementing r & i.


...

T5 BertScore F1: 0.81
8391
3
eu criterion shall be based on the best product available on the community market in term of environmental performance. it shall correspond indicatively to the best 10-20 % of the product available on the community market. eu criterion must be effective, proportionate and dissuasive.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
16042
4
judgment given in a state bound by this convention shall be enforced in another state bound by this convention. no charge, duty or fee calculated by reference to the value of the matter at issue shall be levied in the state in which enforcement is sought. the convention shall be open for signature by the european community. it shall enter into force on the first day of the third month following the deposit of the instrument of accession.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.79
0.8053786158561707
